## Variational Quantum Boltzmann Machine - Qiskit Implementation 

### Imports

In [1]:
# General Imports
import numpy as np
from scipy.linalg import expm

# Circuit Imports
from qiskit.circuit.library import RealAmplitudes, EfficientSU2
from qiskit.circuit import Parameter

# Operator Imports
from qiskit.aqua.operators import I, Z, StateFn, CircuitStateFn, SummedOp
# from qiskit.aqua.operators.gradients import NaturalGradient

# Additional Imports
from qiskit.quantum_info import state_fidelity, partial_trace, Statevector
from qiskit.aqua.components.optimizers import SPSA, CG, ADAM, COBYLA

## Gibbs state preparation using VarQITE

### Define the system parameters and initialize an Ansatz

$\rho^{Gibbs} = \frac{e^H/{k_BT}}{Z}$

Given:
    
    class BM -> H
    H -> f(x)
    
    f(x) -> L(x)
    d/dx L(x)
    

In [2]:
# Temperature
k_BT = 1

# Evolution time
t =  1/(2*k_BT)

# Define the model Hamiltonian 
H = SummedOp([0.3 * Z^Z^ I^I, 0.2 * Z^I^ I^I,  0.5 * I^Z^ I^I]) 

### Define the system parameters and initialize an Ansatz

$\rho^{Gibbs} = \frac{e^H/{k_BT}}{Z}$

In [3]:
# Instantiate the model ansatz
depth = 1
entangler_map = [[i+1, i] for i in range(H.num_qubits - 1)]
ansatz = EfficientSU2(4, reps=depth, entanglement = entangler_map)
qr = ansatz.qregs[0]
for i in range(int(len(qr)/2)):
    ansatz.cx(qr[i], qr[i+int(len(qr)/2)])
    
# Initialize the Ansatz parameters
param_values_init = np.zeros(2* H.num_qubits * (depth + 1))
for j in range(2 * H.num_qubits * depth, int(len(param_values_init) - H.num_qubits - 2)):
    param_values_init[int(j)] = np.pi/2.
    

#### Initial State

The Ansatz $|\psi\left(\omega\left(\tau\right)\right)\rangle$ is initialized such that the first two qubits are in a maximally mixed state.

In [4]:
print('Initial parameters ', param_values_init)

# Initial State

print('\n Circuit ', ansatz.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values_init))))

print('\n Full statevector ', CircuitStateFn(ansatz.assign_parameters \
                                          (dict(zip(ansatz.ordered_parameters, param_values_init)))).eval().primitive.data)

print('\n Maximally mixed state', partial_trace(CircuitStateFn(ansatz.assign_parameters\
                        (dict(zip(ansatz.ordered_parameters, param_values_init)))).eval().primitive.data, [0, 1]).data)

Initial parameters  [0.         0.         0.         0.         0.         0.
 0.         0.         1.57079633 1.57079633 0.         0.
 0.         0.         0.         0.        ]

 Circuit       ┌─────────┐┌─────────┐┌───┐┌──────────┐┌─────────┐                       »
q_0: ┤ RY(0.0) ├┤ RZ(0.0) ├┤ X ├┤ RY(pi/2) ├┤ RZ(0.0) ├───────────────────────»
     ├─────────┤├─────────┤└─┬─┘└──┬───┬───┘├─────────┴┐┌─────────┐           »
q_1: ┤ RY(0.0) ├┤ RZ(0.0) ├──■─────┤ X ├────┤ RY(pi/2) ├┤ RZ(0.0) ├───────────»
     ├─────────┤├─────────┤        └─┬─┘    └──┬───┬───┘├─────────┤┌─────────┐»
q_2: ┤ RY(0.0) ├┤ RZ(0.0) ├──────────■─────────┤ X ├────┤ RY(0.0) ├┤ RZ(0.0) ├»
     ├─────────┤├─────────┤                    └─┬─┘    ├─────────┤├─────────┤»
q_3: ┤ RY(0.0) ├┤ RZ(0.0) ├──────────────────────■──────┤ RY(0.0) ├┤ RZ(0.0) ├»
     └─────────┘└─────────┘                             └─────────┘└─────────┘»
«               
«q_0: ──■───────
«       │       
«q_1: ──┼────■──
«     ┌─┴─┐  │  


#### Let's define the target observable consisting of the Ansatz and the Hamiltonian

$$ \langle \psi\left(\omega\left(\tau\right)\right)|H|\psi\left(\omega\left(\tau\right)\right)\rangle $$

In [5]:
# Define the Hamiltonian as observable w.r.t. the wavefunction generated by the Ansatz   
# Use statevector simulation
ansatz_op = StateFn(ansatz)
op = ~StateFn(H) @ ansatz_op

print('\n Energy expectation value of the initial state ', op.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values_init))).eval())


 Energy expectation value of the initial state  0j


#### Target state

$\rho^{target} = \frac{e^{H\otimes I}/{k_BT}}{Z}$

In [6]:
# Compute the density matrix corresponding to the target Gibbs state
h_mat = H.to_matrix()
gibbs_target = expm(-h_mat*t) / np.trace(expm(-h_mat*t))
gibbs_target = partial_trace(gibbs_target, [0, 1]).data

print('Target state ', gibbs_target)

Target state  [[0.14517971+0.j 0.        +0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.32310338+0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.23936087+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.        +0.j 0.29235603+0.j]]


#### Gradient

Given a parameterized quantum state `|ψ(θ)〉 = V(θ)|ψ〉` with input state `|ψ〉` and parametrized Ansatz `V(θ)`, and an Operator `O(ω)` we want to compute 
``` math
d⟨ψ(θ)|O(ω)|ψ(θ)〉/ dθ
```

In [7]:
param_dict = dict(zip(ansatz.ordered_parameters, np.random.rand(len(ansatz.ordered_parameters))))

'''
Note that this is a simple implementation that only works for Pauli rotations 
and does not account for product or chain rules.
'''

def gradient(op, parameter_dict):
    gradient = []
    for param in list(parameter_dict.keys()):
        parameter_dict[param] += np.pi/2
        p_shift = op.assign_parameters(parameter_dict).eval()
        parameter_dict[param] -= np.pi
        m_shift = op.assign_parameters(parameter_dict).eval()
        parameter_dict[param] += np.pi /2
        gradient += [0.5 * (p_shift - m_shift)]
    return gradient

print('Gradient', gradient(op, param_dict))

Gradient [(-0.48038510085152175-6.249999999999999e-17j), (0.007780004122371363+1.2500000000000006e-17j), (-0.01082636117105814+1.45e-17j), (-0.23462526491553787+1.4e-17j), (0.06751958132344593+8.599999999999999e-17j), (0.008934873448531638+6.5e-17j), (-0.006122188393922118+6.3e-17j), (-0.0049871056488964005+6.85e-17j), (-0.32760193918138564+9.999999999999978e-19j), (-0.12367730309830631-1.0000000000000002e-17j), (0.034265129096000754+3.5000000000000014e-18j), (-0.10294103569294616-1.75e-17j), (-8.326672684688674e-17+4.7e-17j), (2.7755575615628914e-17-6.500000000000001e-18j), 5.0499999999999995e-17j, (-2.7755575615628914e-17+4.9999999999999996e-18j)]


#### Quantum Fisher Information (QFI)

The `QFI` is a metric tensor which is representative for the representation capacity of a parameterized quantum state `|ψ(θ)〉 = V(θ)|ψ〉` generated by an input state `|ψ〉` and a parametrized Ansatz `V(θ)`.

The entries of the `QFI` for a pure state reads
```
[QFI]kl= Re[〈∂kψ|∂lψ〉−〈∂kψ|ψ〉〈ψ|∂lψ〉] * 0.25.
``` 

In [8]:
'''
Implementation according to https://arxiv.org/pdf/2008.06517.pdf
Note that this is a simple implementation that only works for Pauli rotations 
and does not account for product or chain rules.
'''

def QFI(state_op, parameter_dict):
    meas = ~state_op.assign_parameters(parameter_dict)
    qfi = np.zeros((len(parameter_dict), len(parameter_dict)))
    for i, param_i in enumerate(list(parameter_dict.keys())):
        for j, param_j in enumerate(list(parameter_dict.keys())):
            parameter_dict[param_i] += np.pi /2
            parameter_dict[param_j] += np.pi /2
            qfi_op = meas @ state_op.assign_parameters(parameter_dict)
            qfi[i, j] += qfi_op.eval() / 4
            parameter_dict[param_j] -= np.pi
            qfi_op = meas @ state_op.assign_parameters(parameter_dict)
            qfi[i, j] -= qfi_op.eval() / 4
            parameter_dict[param_i] -= np.pi
            qfi_op = meas @ state_op.assign_parameters(parameter_dict)
            qfi[i, j] += qfi_op.eval() / 4
            parameter_dict[param_j] += np.pi
            qfi_op = meas @ state_op.assign_parameters(parameter_dict)
            qfi[i, j] -= qfi_op.eval() / 4
            parameter_dict[param_i] += np.pi/2
            parameter_dict[param_j] -= np.pi/2
    return(qfi)
            
            
            
print('QFI', QFI(ansatz_op, param_dict))

<ipython-input-8-a5ebe029e27d>:15: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] += qfi_op.eval() / 4
<ipython-input-8-a5ebe029e27d>:18: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] -= qfi_op.eval() / 4
<ipython-input-8-a5ebe029e27d>:21: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] += qfi_op.eval() / 4
<ipython-input-8-a5ebe029e27d>:24: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] -= qfi_op.eval() / 4


QFI [[-5.00000000e-01 -2.77555756e-17  0.00000000e+00 -2.77555756e-17
   8.32667268e-17 -5.55111512e-17 -5.55111512e-17  0.00000000e+00
  -3.20049529e-01  1.20021412e-02  2.16481955e-03  9.49763867e-04
  -6.70716879e-02 -1.94909999e-02 -5.53279785e-04 -2.41525120e-03]
 [-2.77555756e-17 -5.00000000e-01  0.00000000e+00 -2.77555756e-17
   2.77555756e-17  2.77555756e-17  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -2.44353189e-01  6.35362775e-03  2.78750534e-03
   2.77555756e-17 -5.72050256e-02 -1.62384610e-03 -7.08863102e-03]
 [ 0.00000000e+00  0.00000000e+00 -5.00000000e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -2.77555756e-17  2.77555756e-17
   0.00000000e+00  0.00000000e+00 -1.41156701e-02  1.54051063e-03
  -5.55111512e-17 -5.55111512e-17 -8.97416104e-04 -3.91752125e-03]
 [-2.77555756e-17 -2.77555756e-17  0.00000000e+00 -5.00000000e-01
   2.77555756e-17  0.00000000e+00 -2.77555756e-17  8.32667268e-17
  -2.77555756e-17  5.55111512e-17  8.32667268e-17 -4.24770357e-03
  -

#### Quantum Natural Gradient (QNG)

The `QNG` combines the state resp. probability gradients and the metric tensor which is representative for the representation capabilities of a parameterized quantum state `|ψ(θ)〉 = V(θ)|ψ〉` generated by an input state `|ψ〉` and a parametrized Ansatz `V(θ)`.

The `QNG` reads
```
QNG = QFI^-1 d⟨ψ(θ)|O(ω)|ψ(θ)〉/ dθ
```
Now, if `O(ω) = H` then QNG is equivalent to the propagation rule applied in VarQITE.

In [9]:
def QNG(op, state_op, parameter_dict):
    grad = gradient(op, parameter_dict)
    qfi = QFI(state_op, parameter_dict)
    alpha = 1e-8
    # Use regularization
    qng, _, _, _ = np.linalg.lstsq(qfi + alpha * np.diag(qfi), grad, rcond=None)
    return qng
    

### Define the parameter propagation rule according to McLachlan's variational principle

In [12]:
'''
The following implementation is based on the following PR https://github.com/Qiskit/qiskit-aqua/pull/1293
'''

def get_gibbs_state_params(op, ansatz, param_values, time_steps):

    # Convert the operator that holds the Hamiltonian and ansatz into a NaturalGradient operator 
    # nat_grad = NaturalGradient().convert(op, ansatz.ordered_parameters, method = 'lin_comb', regularization = 'ridge')
    # Propagate the Ansatz parameters step by step according to the explicit Euler method
    for step in time_steps:
        param_dict = dict(zip(ansatz.ordered_parameters, param_values))
        # nat_grad_result = np.real(nat_grad.assign_parameters(param_dict).eval())
        nat_grad_result = np.real(QNG(op, ansatz_op, param_dict))
        param_values = list(np.subtract(param_values, t/num_time_steps * nat_grad_result))
    return param_values


### Run the parameter propagation

In [13]:
# Define the discretization grid of the time steps
num_time_steps = 10
time_steps = np.linspace(0, t, num_time_steps)
param_values = get_gibbs_state_params(op, ansatz, param_values_init, time_steps)
    
print('Final parameter values', param_values)

<ipython-input-8-a5ebe029e27d>:15: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] += qfi_op.eval() / 4
<ipython-input-8-a5ebe029e27d>:18: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] -= qfi_op.eval() / 4
<ipython-input-8-a5ebe029e27d>:21: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] += qfi_op.eval() / 4
<ipython-input-8-a5ebe029e27d>:24: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] -= qfi_op.eval() / 4


Final parameter values [-1.175909530049918, 0.37777127988870435, 3.9836840995699735e-10, 3.7443554837116214e-10, 0.6774145304868413, 0.5163793969835995, -0.3226838262048864, -0.3226847411481746, 2.2289309272402176, 1.5158619122489765, 3.871996961402439e-10, 3.7540706364605423e-10, 0.12176363303934118, 0.1301222937295763, -0.32268588846005003, -0.3226858928164995]


#### Check the fidelity between trained and target state

In [14]:
# Compute the density matrix corresponding to the final Gibbs state    
gibbs_state = Statevector.from_instruction(ansatz.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values))))
gibbs_state = partial_trace(gibbs_state, [0, 1])
print('Gibbs state ', np.around(gibbs_state.data, 3))

print('Target state ', np.around(gibbs_target.data, 3))

# Evaluate the fidelity between the trained and the target state
fidelity = state_fidelity(np.around(gibbs_target.data, 3), np.around(gibbs_state.data, 3), validate=False)

print('Fidelity between trained and target state ', fidelity)

Gibbs state  [[0.491-0.j 0.   -0.j 0.   +0.j 0.   +0.j]
 [0.   +0.j 0.149-0.j 0.   -0.j 0.   -0.j]
 [0.   -0.j 0.   +0.j 0.184+0.j 0.   +0.j]
 [0.   -0.j 0.   +0.j 0.   -0.j 0.176+0.j]]
Target state  [[0.145+0.j 0.   +0.j 0.   +0.j 0.   +0.j]
 [0.   +0.j 0.323+0.j 0.   +0.j 0.   +0.j]
 [0.   +0.j 0.   +0.j 0.239+0.j 0.   +0.j]
 [0.   +0.j 0.   +0.j 0.   +0.j 0.292+0.j]]
Fidelity between trained and target state  0.8512003018378689


## Train a generative QBM 
More explicitly, we train here a fully visible, diagonal, generative QBM using gradient-free optimization.

### Initialize a parameterized Hamiltonian and the target PDF

In [15]:
a = Parameter('a')    
b = Parameter('b')
c = Parameter('c')  

# Define the model Hamiltonian with parameters
H = SummedOp([a * Z ^ Z ^ I ^ I, b * Z ^I ^ I ^ I, c * I ^ Z^ I ^ I]) 

# Define the target PDF
p_target = [0.5, 0, 0, 0.5] 

#### Define the loss function and the optimizer

In [16]:
# Define the loss function
def loss(H_coeffs):
    H_op = H.assign_parameters(dict(zip([a, b, c], np.real(H_coeffs))))
    
    #Combine the measurement and ansatz operator
    op = ~StateFn(H_op) @ ansatz_op
    
    #Prepare the Gibbs state
    param_values = get_gibbs_state_params(op, ansatz, param_values_init, time_steps)
    p_qbm = ansatz_op.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values))).eval().primitive
    p_qbm = np.diag(partial_trace(p_qbm, [0, 1]).data)
    print('Trained probability ', p_qbm)
    loss_fn = -np.sum(np.multiply(p_target, np.log(p_qbm)))
#     print(loss_fn)
    return np.real(loss_fn)

### Customize
Try different optimizers

In [17]:
optimizer = COBYLA(maxiter = 50)

### Train the QBM

In [ ]:
result = optimizer.optimize(3, loss, initial_point = ([-2., .2, .5]))
print('Trained parameters ', result[0])

<ipython-input-8-a5ebe029e27d>:15: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] += qfi_op.eval() / 4
<ipython-input-8-a5ebe029e27d>:18: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] -= qfi_op.eval() / 4
<ipython-input-8-a5ebe029e27d>:21: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] += qfi_op.eval() / 4
<ipython-input-8-a5ebe029e27d>:24: ComplexWarning: Casting complex values to real discards the imaginary part
  qfi[i, j] -= qfi_op.eval() / 4


Trained probability  [0.02265963+8.78065566e-20j 0.41393144+4.20450968e-19j
 0.55819431-2.41032652e-19j 0.00521463-2.33215054e-20j]
Trained probability  [0.13789179-3.69738271e-19j 0.29974951-1.28539151e-17j
 0.52598523-2.92417754e-19j 0.03637347-5.14157517e-19j]
Trained probability  [0.267414  +1.71659687e-18j 0.63630374+1.82458354e-18j
 0.07947283+6.86309983e-19j 0.01680943-2.28495231e-19j]
Trained probability  [0.06054083-1.69448844e-18j 0.37470015+5.34002503e-18j
 0.43551687+8.82662023e-19j 0.12924215-6.69423467e-19j]
Trained probability  [0.54934605+1.22139685e-18j 0.03228803+5.60379062e-20j
 0.39584214+2.15420857e-19j 0.02252377+6.67472932e-20j]
Trained probability  [0.73731816-2.60745161e-17j 0.00441193+2.30545531e-20j
 0.24269991+3.19775983e-18j 0.01557   +5.74655140e-20j]
Trained probability  [0.39696385+2.39240453e-20j 0.033039  +4.05459455e-20j
 0.5030389 +4.25575012e-19j 0.06695825-2.88164785e-19j]
Trained probability  [0.30597034+3.67858289e-19j 0.01758813+4.51077181e-20j


In [ ]:
#Construct the Hamiltonian with the final parameterw
H_op = H.assign_parameters(dict(zip([a, b, c], [-1.29810568, -0.04292338,  0.1362023])))

#Combine the measurement and ansatz operator
op = ~StateFn(H_op) @ ansatz_op

#Prepare the Gibbs state
param_values = get_gibbs_state_params(op, ansatz, param_values_init, time_steps)

# Get the sampling probabilities
p_qbm = ansatz_op.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values))).eval().primitive
p_qbm = np.diag(partial_trace(p_qbm, [0, 1]).data)

# Evaluate the l1 norm between the trained and the target state
norm = np.linalg.norm(p_target-p_qbm, ord = 1)

print('L1-norm between trained and target distributions ', norm)

In [ ]:
print(p_qbm)